#  Scraping sur Ubuy

Ce notebook permet de scraper des informations détaillées sur des produits depuis le site **Ubuy** en utilisant Selenium avec _undetected_chromedriver_.
Le script réalise les opérations suivantes :

- **Initialisation du navigateur** en mode "undetected" pour contourner les mesures anti-bot.
- **Gestion de CAPTCHA** : Si un CAPTCHA est détecté, l'utilisateur est invité à le résoudre manuellement.
- **Scraping des détails produits** : Extraction des spécifications techniques à partir des pages produit.
- **Navigation multi-pages** : Parcours des pages d'un catalogue de produits.
- **Sauvegarde des données** dans un fichier CSV, avec gestion de la version (numérotation des scrapes).

Les bibliothèques principales utilisées sont :
- `selenium` et `undetected_chromedriver` pour automatiser le navigateur.
- `BeautifulSoup` pour le parsing du HTML.
- `csv`, `os` et `datetime` pour la sauvegarde des données.
- `ThreadPoolExecutor` pour exécuter des tâches en parallèle (lors du scraping des détails produits).
- `logging` pour le suivi des opérations.

Chaque fonction du script est commentée en détail afin de faciliter sa compréhension.


In [ ]:
import os
import csv
import time
import random
import logging
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor, as_completed
import undetected_chromedriver as uc


## Configuration du Logging

Nous utilisons le module `logging` pour afficher des messages informatifs sur l'état d'exécution du script.
Cela permet de suivre l'initialisation du driver, le scraping de chaque page ou produit, ainsi que les éventuelles erreurs.


In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


## Fonction `get_random_user_agent`

Cette fonction retourne un User-Agent choisi aléatoirement à partir d'une liste prédéfinie.
Cela permet de modifier régulièrement le User-Agent envoyé dans les requêtes afin d'éviter la détection par le site.


In [ ]:
def get_random_user_agent():
    """Retourne un user-agent choisi aléatoirement pour éviter la détection."""
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
    ]
    return random.choice(user_agents)


## Fonction `get_driver`

Cette fonction initialise et retourne une instance d'Undetected ChromeDriver.
Les options du navigateur sont configurées pour :
- Lancer le navigateur en mode non-headless (pour le debugging, vous pouvez passer en headless en modifiant l'option).
- Définir la taille de la fenêtre, désactiver certaines optimisations susceptibles de trahir l'automatisation et ajouter un User-Agent aléatoire.
- Utiliser le _ChromeDriverManager_ pour gérer automatiquement l'installation du driver.

En cas d'erreur lors de l'initialisation, un message d'erreur est loggé et l'exception est propagée.


In [ ]:
def get_driver():
    """Initialise une instance d'Undetected ChromeDriver."""
    try:
        logging.info("Initializing ChromeDriver...")
        options = uc.ChromeOptions()
        options.headless = False  # Mettre sur True pour exécuter en arrière-plan
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920x1080")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(f"--user-agent={get_random_user_agent()}")

        service = Service(ChromeDriverManager().install())
        driver = uc.Chrome(service=service, options=options)
        logging.info("ChromeDriver initialized successfully!")
        return driver
    except Exception as e:
        logging.error(f"Failed to initialize ChromeDriver: {e}")
        raise


## Fonction `handle_captcha`

Cette fonction interrompt l'exécution du script lorsqu'un CAPTCHA est détecté,
permettant ainsi à l'utilisateur de résoudre manuellement le CAPTCHA avant de reprendre l'exécution.


In [ ]:
def handle_captcha(driver):
    """Met en pause l'exécution pour permettre à l'utilisateur de résoudre le CAPTCHA."""
    logging.info("CAPTCHA detected. Please solve it manually.")
    input("Press Enter to continue after solving the CAPTCHA...")
    logging.info("Resuming script execution...")


## Fonction `scrape_product_details`

Cette fonction effectue le scraping d'une page produit en suivant ces étapes :

1. **Chargement de la page** : Le driver navigue vers l'URL du produit.
2. **Délais aléatoires** : Un délai aléatoire est appliqué pour simuler un comportement humain.
3. **Détection du CAPTCHA** : Si un CAPTCHA est présent, l'utilisateur est invité à le résoudre.
4. **Attente de chargement** : Le script attend que la table de spécifications (dans les sections `#additional-info` ou `#technical-info`) soit présente.
5. **Parsing du contenu** : BeautifulSoup est utilisé pour extraire les spécifications présentes dans la table.
6. **Retour** : Un dictionnaire contenant les spécifications est retourné.

En cas d'erreur, un dictionnaire vide est retourné et l'erreur est loggée.


In [ ]:
def scrape_product_details(driver, product_url):
    """Scrape les spécifications détaillées d'un produit à partir de sa page."""
    try:
        logging.info(f"Scraping product: {product_url}")
        driver.get(product_url)
        time.sleep(random.uniform(2, 5))  # Délai aléatoire

        # Vérifie la présence d'un CAPTCHA
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "iframe[src*='captcha']"))
            )
            handle_captcha(driver)  # Pause pour résolution manuelle du CAPTCHA
        except:
            logging.info("No CAPTCHA detected. Proceeding with scraping...")

        # Attente que la table de spécifications soit chargée
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div#additional-info table, div#technical-info table"))
        )

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        specs = {}

        # Extraction des spécifications depuis les tables
        spec_tables = soup.select("div#additional-info table, div#technical-info table")
        for table in spec_tables:
            for row in table.find_all("tr"):
                cols = row.find_all("td")
                if len(cols) == 2:
                    key = cols[0].text.strip()
                    value = cols[1].text.strip()
                    specs[key] = value

        return specs
    except Exception as e:
        logging.error(f"Error scraping {product_url}: {e}")
        return {}


## Fonction `get_next_scrape_number`

Cette fonction détermine le numéro du prochain scrape en vérifiant les fichiers existants dans le dossier de sortie.
Cela permet de versionner les fichiers CSV et d'éviter de les écraser lors de sauvegardes successives.


In [ ]:
def get_next_scrape_number(output_dir, category):
    """Détermine le prochain numéro de scrape pour la version d'un fichier CSV."""
    scrape_number = 1
    for filename in os.listdir(output_dir):
        if filename.startswith(f"{category}_") and filename.endswith(".csv"):
            try:
                current_number = int(filename.split('_scrape')[-1].split('.')[0])
                if current_number >= scrape_number:
                    scrape_number = current_number + 1
            except ValueError:
                continue
    return scrape_number


## Fonction `save_to_csv`

Cette fonction enregistre les données scrapées dans un fichier CSV.
Les principales étapes sont :
- Création d'un répertoire spécifique à la catégorie si nécessaire.
- Détermination du numéro de scrape (pour versionner les fichiers).
- Construction des en-têtes du CSV (incluant les colonnes de base et toutes les clés de spécifications rencontrées).
- Écriture de chaque ligne dans le CSV.


In [ ]:
def save_to_csv(data, category, all_spec_keys):
    """Enregistre les données scrapées dans un fichier CSV."""
    # Création d'un répertoire dédié à la catégorie
    output_dir = os.path.join("data/raw/ubuy", category)
    os.makedirs(output_dir, exist_ok=True)

    today_date = datetime.today().strftime("%Y_%m_%d")
    scrape_number = get_next_scrape_number(output_dir, category)
    filename = f"{category}_{today_date}_scrape{scrape_number}.csv"
    filepath = os.path.join(output_dir, filename)

    # Définition des colonnes : données de base + toutes les clés de spécifications
    fieldnames = ["title", "price", "image_url", "product_url", "Collection Date"] + list(all_spec_keys)

    with open(filepath, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for item in data:
            row = {
                "title": item["title"],
                "price": item["price"],
                "image_url": item["image_url"],
                "product_url": item["product_url"],
                "Collection Date": today_date
            }
            row.update(item["specifications"])
            writer.writerow(row)

    logging.info(f"Data saved to {filepath}")


## Fonction `scrape_ubuy`

Cette fonction orchestre le scraping d'une catégorie de produits sur Ubuy.
Les étapes réalisées sont :

1. **Navigation et gestion des pages** :
   - Le driver charge la page de base.
   - Le script récupère les blocs produits et en extrait les URLs.
   - Si un CAPTCHA est détecté, l'utilisateur est invité à le résoudre.

2. **Scraping concurrent des détails produits** :
   - Pour chaque URL de produit, la fonction `scrape_product_details` est appelée en parallèle (via `ThreadPoolExecutor`).
   - Les clés de spécifications rencontrées sont collectées dans un ensemble `all_spec_keys`.

3. **Navigation vers la page suivante** :
   - Le script recherche le bouton ou l'élément permettant d'accéder à la page suivante.
   - Si une page suivante est trouvée, l'URL est mise à jour, sinon le scraping s'arrête.

4. **Fermeture du driver** :
   - Une fois le scraping terminé (ou en cas d'erreur), le driver est fermé.

La fonction retourne les items scrapés ainsi que l'ensemble des clés de spécifications pour la sauvegarde.


In [ ]:
def scrape_ubuy(driver, base_url, max_pages):
    """Scrape les données produit sur plusieurs pages d'Ubuy."""
    scraped_items = []
    all_spec_keys = set()

    try:
        current_page = 1
        current_url = base_url

        while current_page <= max_pages:
            logging.info(f"Scraping page {current_page}: {current_url}")
            driver.get(current_url)
            time.sleep(random.uniform(3, 6))  # Délai aléatoire

            # Vérifie la présence d'un CAPTCHA
            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe[src*='captcha']"))
                )
                handle_captcha(driver)
            except:
                logging.info("No CAPTCHA detected. Proceeding with scraping...")

            # Attendre que les produits soient chargés
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.product-card"))
                )
            except Exception as e:
                logging.error("No products found. Page may have changed.")
                break

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            product_blocks = soup.find_all('div', class_='product-card')

            if not product_blocks:
                logging.info("No products found. Exiting scraping.")
                break

            # Récupérer les URLs des produits
            product_urls = []
            for product in product_blocks:
                link_element = product.find('a', class_='product-img')
                if link_element and "href" in link_element.attrs:
                    product_url = link_element['href']
                    full_product_url = f"https://www.ubuy.ma{product_url}" if product_url.startswith('/') else product_url
                    product_urls.append(full_product_url)

            # Scraping concurrent des détails produits
            with ThreadPoolExecutor(max_workers=5) as executor:
                future_to_url = {executor.submit(scrape_product_details, driver, url): url for url in product_urls}
                for future in as_completed(future_to_url):
                    url = future_to_url[future]
                    try:
                        specifications = future.result()
                        all_spec_keys.update(specifications.keys())

                        # Rechercher le bloc produit correspondant pour récupérer title, price, image_url
                        for product in product_blocks:
                            link_element = product.find('a', class_='product-img')
                            if link_element and "href" in link_element.attrs:
                                product_url = link_element['href']
                                full_product_url = f"https://www.ubuy.ma{product_url}" if product_url.startswith('/') else product_url
                                if full_product_url == url:
                                    title = product.find('h3', class_='product-title').text.strip() if product.find('h3', class_='product-title') else "No title"
                                    price = product.find('p', class_='product-price').text.strip() if product.find('p', class_='product-price') else "No price"
                                    image_url = product.find('img')['src'] if product.find('img') else "No image"

                                    scraped_items.append({
                                        "title": title,
                                        "price": price,
                                        "image_url": image_url,
                                        "product_url": full_product_url,
                                        "specifications": specifications
                                    })
                                    break
                    except Exception as e:
                        logging.error(f"Error processing {url}: {e}")

            # Recherche du lien vers la page suivante
            next_page_element = soup.find('li', class_='page-item', title=str(current_page + 1))
            if next_page_element:
                next_button = next_page_element.find('button', class_='page-link')
                if next_button and "data-pageno" in next_button.attrs:
                    next_page_number = next_button['data-pageno']
                    current_url = f"{base_url}&page={next_page_number}"
                    current_page += 1
                    time.sleep(random.uniform(3, 7))
                else:
                    logging.info("No more pages found.")
                    break
            else:
                logging.info("No more pages found.")
                break

    except Exception as e:
        logging.error(f"Error during scraping: {e}")
    finally:
        driver.quit()

    return scraped_items, all_spec_keys


## Bloc Principal d'Exécution

Dans ce bloc, nous définissons un dictionnaire `categories` qui associe à chaque catégorie :
- Une URL de base pour démarrer le scraping.
- Le nombre maximum de pages à parcourir.

Pour chaque catégorie :
1. Nous initialisons le driver avec `get_driver()`.
2. Nous appelons `scrape_ubuy` pour récupérer les données produits.
3. Si des données ont été récupérées, nous les sauvegardons dans un fichier CSV à l'aide de `save_to_csv`.

En cas d'erreur, le message sera loggé.


In [ ]:
if __name__ == "__main__":
    try:
        logging.info("Starting script...")
        categories = {
            "graphics_cards": ("https://www.ubuy.ma/en/search/?ref_p=ser_tp&q=graphics+cards", 8),
            "laptops": ("https://www.ubuy.ma/en/category/laptops-21457", 8),
            "monitors": ("https://www.ubuy.ma/en/search/?q=computer%20monitor", 8),
            "smart_watches": ("https://www.ubuy.ma/en/search/?ref_p=ser_tp&q=smart+watch", 8)
        }

        for category, (base_url, max_pages) in categories.items():
            logging.info(f"Scraping {category}...")
            driver = get_driver()
            scraped_data, all_spec_keys = scrape_ubuy(driver, base_url, max_pages)

            if scraped_data:
                save_to_csv(scraped_data, category, all_spec_keys)
            else:
                logging.info(f"No data scraped for {category}.")
    except Exception as e:
        logging.error(f"An error occurred: {e}")


## Conclusion

Ce notebook démontre comment utiliser Selenium et Undetected Chromedriver pour scraper des informations détaillées depuis Ubuy.
Chaque fonction a été commentée en détail pour expliquer son rôle et son fonctionnement :

- **get_random_user_agent** : Retourne un User-Agent aléatoire.
- **get_driver** : Initialise et configure le driver pour contourner la détection.
- **handle_captcha** : Permet à l'utilisateur de résoudre manuellement un CAPTCHA.
- **scrape_product_details** : Extrait les spécifications d'un produit depuis sa page.
- **get_next_scrape_number** : Gère la version des fichiers de sortie.
- **save_to_csv** : Enregistre les données dans un fichier CSV.
- **scrape_ubuy** : Parcourt plusieurs pages d'une catégorie et scrape les produits.

Avant d'exécuter ce notebook, assurez-vous d'avoir installé les dépendances requises (par exemple, via `pip install selenium undetected-chromedriver webdriver-manager bs4`).

Bonne utilisation et bon scraping !
